# MCP + LangGraph 핸즈온 튜토리얼

- 작성자: [테디노트](https://youtube.com/c/teddynote)
- 강의: [패스트캠퍼스 RAG 비법노트](https://fastcampus.co.kr/data_online_teddy)

**참고자료**
- https://modelcontextprotocol.io/introduction
- https://github.com/langchain-ai/langchain-mcp-adapters

## 환경설정

아래 설치 방법을 참고하여 `uv` 를 설치합니다.

**uv 설치 방법**

```bash
# macOS/Linux
curl -LsSf https://astral.sh/uv/install.sh | sh

# Windows (PowerShell)
irm https://astral.sh/uv/install.ps1 | iex
```

**의존성 설치**

```bash
uv pip install -r requirements.txt
```

환경변수를 가져옵니다.

In [1]:
from dotenv import load_dotenv
from langchain_teddynote import logging

load_dotenv(override=True)
logging.langsmith("LangGraph MCP Adapters HandsOn")

LangSmith 추적을 시작합니다.
[프로젝트명]
LangGraph MCP Adapters


## MultiServerMCPClient

사전에 `mcp_server_local.py` 를 실행해둡니다. 터미널을 열고 가상환경이 활성화 되어 있는 상태에서 서버를 실행해 주세요.

> 명령어
```bash
source .venv/bin/activate
python mcp_server_local.py
```

`async with` 로 일시적인 Session 연결을 생성 후 해제

In [ ]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_teddynote.messages import ainvoke_graph, astream_graph
from langchain_anthropic import ChatAnthropic

model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

async with MultiServerMCPClient(
    {
        "weather": {
            # 서버의 포트와 일치해야 합니다.(8005번 포트)
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
) as client:
    print(client.get_tools())
    agent = create_react_agent(model, client.get_tools())
    answer = await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x10b686160>)]

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울의 날씨 정보를 확인해 드리겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울
🔄 Node: agent 🔄
- - - - - - - - - - - - - -

다음의 경우에는 session 이 닫혔기 때문에 도구에 접근할 수 없는 것을 확인할 수 있습니다.

In [4]:
await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울의 날씨 정보를 확인해 드리겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Error: ClosedResourceError()
 Please fix your mistakes.
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
죄송합니다. 서울의 날씨 정보를 가져오는 데 오류가 발생했습니다. 다시 시도해 보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Error: ClosedResourceError()
 Please fix your mistakes.
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
죄송합니다만, 현재 날씨 정보 서비스에 기술적인 문제가 있는 것 같습니다. 잠시 후에 다시 시도해 주시거나, 기상청 웹사이트나 다른 날씨 앱을 통해 서울의 날씨를 확인해 보시는 것을 권장드립니다.

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-a6cf4a58-4c7b-40f1-8baf-cde560dee7a2', usage_metadata={'input_tokens': 0, 'output_tokens': 120, 'total_tokens': 120, 'input_token_details': {}}),
 'metadata': {'langgraph_step': 5,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:2ab521ec-3c27-60c7-cec6-14ce2055186d',
  'checkpoint_ns': 'agent:2ab521ec-3c27-60c7-cec6-14ce2055186d',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': None,
  'ls_max_tokens': 1024}}

이제 그럼 Async Session 을 유지하며 도구에 접근하는 방식으로 변경해 보겠습니다.

In [5]:
# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "weather": {
            "url": "http://localhost:8005/sse",
            "transport": "sse",
        }
    }
)


# 2. 명시적으로 연결 초기화 (이 부분이 필요함)
# 초기화
await client.__aenter__()

# 이제 도구가 로드됨
print(client.get_tools())  # 도구가 표시됨

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x10dba58a0>)]


langgraph 의 에이전트를 생성합니다.

In [6]:
# 에이전트 생성
agent = create_react_agent(model, client.get_tools())

그래프를 실행하여 결과를 확인합니다.

In [7]:
await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울의 날씨를 확인해 드리겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
It's always Sunny in 서울
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
서울의 날씨는 현재 맑고 화창합니다. "It's always Sunny in 서울" - 서울은 항상 맑다고 합니다.

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-09487bd1-5f00-421e-a025-1dca4782c1d9', usage_metadata={'input_tokens': 0, 'output_tokens': 55, 'total_tokens': 55, 'input_token_details': {}}),
 'metadata': {'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:4f2608a9-033a-eb97-d598-0e314dbcabe3',
  'checkpoint_ns': 'agent:4f2608a9-033a-eb97-d598-0e314dbcabe3',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': None,
  'ls_max_tokens': 1024}}

## Stdio 통신 방식

Stdio 통신 방식은 로컬 환경에서 사용하기 위해 사용합니다.

- 통신을 위해 표준 입력/출력 사용

In [2]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langgraph.prebuilt import create_react_agent
from langchain_mcp_adapters.tools import load_mcp_tools
from langchain_anthropic import ChatAnthropic

# Anthropic의 Claude 모델 초기화
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# StdIO 서버 파라미터 설정
# - command: Python 인터프리터 경로
# - args: 실행할 MCP 서버 스크립트
server_params = StdioServerParameters(
    command="/Users/teddy/Dev/github/langgraph-mcp-adapters/.venv/bin/python",
    args=["mcp_server_local.py"],
)

# StdIO 클라이언트를 사용하여 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드
        tools = await load_mcp_tools(session)
        print(tools)

        # 에이전트 생성
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        await astream_graph(agent, {"messages": "서울의 날씨는 어떠니?"})

[StructuredTool(name='get_weather', description='\n    Get current weather information for the specified location.\n\n    This function simulates a weather service by returning a fixed response.\n    In a production environment, this would connect to a real weather API.\n\n    Args:\n        location (str): The name of the location (city, region, etc.) to get weather for\n\n    Returns:\n        str: A string containing the weather information for the specified location\n    ', args_schema={'properties': {'location': {'title': 'Location', 'type': 'string'}}, 'required': ['location'], 'title': 'get_weatherArguments', 'type': 'object'}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x1157e8cc0>)]


  + Exception Group Traceback (most recent call last):
  |   File "/Users/teddy/Dev/github/langgraph-mcp-adapters/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py", line 3547, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/wn/fz0vz0td5bn8wl7wkrv30x100000gn/T/ipykernel_87574/4211018985.py", line 21, in <module>
  |     async with stdio_client(server_params) as (read, write):
  |                ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/teddy/.local/share/uv/python/cpython-3.12.8-macos-aarch64-none/lib/python3.12/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(value)
  |   File "/Users/teddy/Dev/github/langgraph-mcp-adapters/.venv/lib/python3.12/site-packages/mcp/client/stdio.py", line 148, in stdio_client
  |     anyio.create_task_group() as tg,
  |     ^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/Users/teddy/Dev/github/langgraph-mcp-adapters/.venv/lib/python3.12/site-packages/anyio/_backends/_as

## RAG 를 구축한 MCP 서버 사용

- 파일: `mcp_server_rag.py`

사전에 langchain 으로 구축한 `mcp_server_rag.py` 파일을 사용합니다.

stdio 통신 방식으로 도구에 대한 정보를 가져옵니다. 여기서 도구는 `retriever` 도구를 가져오게 되며, 이 도구는 `mcp_server_rag.py` 에서 정의된 도구입니다. 이 파일은 사전에 서버에서 실행되지 **않아도** 됩니다.

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic
from langchain_teddynote.messages import astream_graph

# Anthropic의 Claude 모델 초기화
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# RAG 서버를 위한 StdIO 서버 파라미터 설정
server_params = StdioServerParameters(
    command="/Users/teddy/Dev/github/langgraph-mcp-adapters/.venv/bin/python",
    args=["./mcp_server_rag.py"],
)

# StdIO 클라이언트를 사용하여 RAG 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드 (여기서는 retriever 도구)
        tools = await load_mcp_tools(session)

        # 에이전트 생성 및 실행
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        await astream_graph(
            agent, {"messages": "삼성전자가 개발한 생성형 AI의 이름을 검색해줘"}
        )


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI에 대한 정보를 검색해보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙

Error in sse_reader: peer closed connection without sending complete message body (incomplete chunked read)


## SSE 방식과 StdIO 방식 혼합 사용

- 파일: `mcp_server_rag.py` 는 StdIO 방식으로 통신
- `langchain-dev-docs` 는 SSE 방식으로 통신

SSE 방식과 StdIO 방식을 혼합하여 사용합니다.

In [10]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# Anthropic의 Claude 모델 초기화
model = ChatAnthropic(
    model_name="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000
)

# 1. 다중 서버 MCP 클라이언트 생성
client = MultiServerMCPClient(
    {
        "document-retriever": {
            "command": "python",
            # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
            "args": ["./mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
        "langchain-dev-docs": {
            # SSE 서버가 8765 포트에서 실행 중인지 확인하세요
            "url": "http://teddynote.io:8765/sse",
            # SSE(Server-Sent Events) 방식으로 통신
            "transport": "sse",
        },
    }
)


# 2. 비동기 컨텍스트 매니저를 통한 명시적 연결 초기화
await client.__aenter__()

Error in sse_reader: 


langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [11]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = (
    "You are a smart agent. "
    "Use `retriever` tool to search on AI related documents and answer questions."
    "Use `langchain-dev-docs` tool to search on langchain / langgraph related documents and answer questions."
    "Answer in Korean."
)
agent = create_react_agent(
    model, client.get_tools(), prompt=prompt, checkpointer=MemorySaver()
)

구축해 놓은 `mcp_server_rag.py` 에서 정의한 `retriever` 도구를 사용하여 검색을 수행합니다.

In [13]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {
        "messages": "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘"
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI 이름을 검색해보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-bb7d986d-28b3-4323-89cb-9fdd8417b491', usage_metadata={'input_tokens': 0, 'output_tokens': 350, 'total_tokens': 350, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 10,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:51bd1936-85dc-4967-36ed-65178af522ee',
  'checkpoint_ns': 'agent:51bd1936-85dc-4967-36ed-65178af522ee',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 1024}}

이번에는 `langchain-dev-docs` 도구를 사용하여 검색을 수행합니다.

In [14]:
config = RunnableConfig(recursion_limit=30, thread_id=1)
await astream_graph(
    agent,
    {"messages": "langgraph-dev-docs 참고해서 self-rag 의 정의에 대해서 알려줘"},
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
LangGraph 문서를 참고하여 Self-RAG의 정의에 대해 알아보겠습니다. 먼저 LangGraph 관련 문서를 검색해보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
["Below is the list of references for the answer to the question.(Ordered by relevance descending):", "1. https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_adaptive_rag/", "2. https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_self_rag/", "3. https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_crag/", "4. https://langchain-ai.github.io/langgraph/tutorials/code_assistant/langgraph_code_assistant/"]
🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Self-RAG에 대한 정의를 찾기 위해 두 번째 링크를 확인해보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Self-RAG






[Skip to content](#self-rag)

**Join us at  [Interrupt: The Agent AI Conference by LangChain](https://interrupt.langchain.com/) on May 13 & 14 in San Franci

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'max_tokens', 'stop_sequence': None}, id='run-2d6c90ba-f43c-46f5-b7cb-2b617685b459', usage_metadata={'input_tokens': 0, 'output_tokens': 1024, 'total_tokens': 1024, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 17,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:c9695a1d-8b32-295e-9bff-1169abfd0930',
  'checkpoint_ns': 'agent:c9695a1d-8b32-295e-9bff-1169abfd0930',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 1024}}

`MemorySaver` 를 사용하여 단기 기억을 유지합니다. 따라서, multi-turn 대화도 가능합니다.

In [15]:
await astream_graph(
    agent, {"messages": "이전의 내용을 bullet point 로 요약해줘"}, config=config
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
# Self-RAG 정의 요약

## 기본 개념
* Self-RAG는 검색된 문서와 생성된 텍스트에 대한 자기 성찰 및 자기 평가를 포함하는 RAG 전략
* 논문 "Self-RAG"(https://arxiv.org/abs/2310.11511)에서 제안된 개념

## 주요 결정 과정
* **검색 여부 결정**
  - 입력: 질문(x) 또는 질문(x)과 생성된 텍스트(y)
  - 출력: yes, no, continue
  
* **문서 관련성 평가**
  - 입력: 질문(x)과 검색된 문서 청크(d)
  - 출력: relevant, irrelevant
  
* **생성 텍스트의 사실성 평가**
  - 입력: 질문(x), 문서 청크(d), 생성된 텍스트(y)
  - 출력: fully supported, partially supported, no support
  
* **생성 텍스트의 유용성 평가**
  - 입력: 질문(x), 생성된 텍스트(y)
  - 출력: 1-5점 척도

## 주요 구성 요소
* **검색기(Retriever)**: 질문 관련 문서 검색
* **문서 평가기(Document Grader)**: 문서의 관련성 평가
* **생성기(Generator)**: 관련 문서 기반 답변 생성
* **환각 평가기(Hallucination Grader)**: 생성된 답변의 사실성 평가
* **답변 평가기(Answer Grader)**: 생성된 답변의 질문 해결 여부 평가
* **질문 재작성기(Question Rewriter)**: 더 나은 검색을 위한 질문 재작성

## 워크플로우
1. 질문 입력 → 문서 검색
2. 문서 관련성 평가 및 필터링
3. 관련 문서 없음 → 질문 재작성 → 다시 검색
4. 관련 문서 있음 → 답변 생성
5. 생성된 답변 평가 (사실성, 유용성)
6. 평가 결과에 따라 최종 답변 제공 또는 프로세스 반복

## 장점
*

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-0f4994e4-e1ea-48e6-9a9a-dbca104daa2a', usage_metadata={'input_tokens': 0, 'output_tokens': 791, 'total_tokens': 791, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 20,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:99d3dec5-ab94-edc4-ce11-52ce1bf0311e',
  'checkpoint_ns': 'agent:99d3dec5-ab94-edc4-ce11-52ce1bf0311e',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 1024}}

## LangChain 에 통합된 도구 + MCP 도구

여기서는 LangChain 에 통합된 도구를 기존의 MCP 로만 이루어진 도구와 함께 사용이 가능한지 테스트 합니다.

In [16]:
from langchain_teddynote.tools.tavily import TavilySearch

# Tavily 검색 도구를 초기화 합니다. (news 타입, 최근 3일 내 뉴스)
tavily = TavilySearch(max_results=3, topic="news", days=3)

# 기존의 MCP 도구와 함께 사용합니다.
tools = client.get_tools() + [tavily]

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [17]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig

prompt = "You are a smart agent with various tools. Answer questions in Korean."
agent = create_react_agent(model, tools, prompt=prompt, checkpointer=MemorySaver())

새롭게 추가한 `tavily` 도구를 사용하여 검색을 수행합니다.

In [18]:
await astream_graph(agent, {"messages": "오늘 뉴스 찾아줘"}, config=config)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
오늘 뉴스를 찾아드리겠습니다. 웹 검색을 통해 최신 뉴스를 검색해보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
[{"url": "https://racingnews365.com/lawson-handed-second-arrogance-evaluation-after-red-bull-axe", "raw_content": "Published Time: 2025-03-29T12:00:00+01:00\nLawson handed second ‘arrogance’ evaluation after Red Bull axe | RacingNews365\nRN365 Podcast Red Bull emergency meeting as Ferrari review disqualifications\nLogin\nHome\nF1 news\nCalendar\nStandings\nHamilton\nFull menu\nHome\nF1 news\nF1 Calendar\nF1 Standings\nInterviews\nVideos\nPodcast\nFormula E\nClose More\nHome\nF1 news\nF1 calendar 2025\nF1 standings 2025\nF1 drivers\nF1 teams\nF1 circuits\nF1 results\nF1 Live\nF1 videos\nF1 podcast\nFormula E\nClose menu\nWelcome at RacingNews365\nBecome part of the largest racing community in the United Kingdom. Create your free account now!\nShare your thoughts and opinions about F1\nWin fantastic prizes\nGet a

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-44e4bb24-9b33-4c05-99fb-a18f29203442', usage_metadata={'input_tokens': 0, 'output_tokens': 976, 'total_tokens': 976, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 3,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:1eabca91-8843-ec33-e706-46ad0d095a2c',
  'checkpoint_ns': 'agent:1eabca91-8843-ec33-e706-46ad0d095a2c',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 1024}}

`retriever` 도구가 원활하게 작동하는 것을 확인할 수 있습니다.

In [19]:
await astream_graph(
    agent,
    {
        "messages": "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 이름을 검색해줘"
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI 이름을 검색해보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번역 업무의 
처리를 지원
∙코드 모델 기반의 AI 코딩 어시스턴트 ‘코드아이(code.i)’는 대화형 인터페이스로 서비스를 제공하며 
사내 소프트웨어 개발에 최적화
∙이미지 

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-a5411745-0513-428a-a421-b9dd3a154ef9', usage_metadata={'input_tokens': 0, 'output_tokens': 581, 'total_tokens': 581, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 8,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:7ab41c2e-4e69-7653-c92b-bacc7f1f151a',
  'checkpoint_ns': 'agent:7ab41c2e-4e69-7653-c92b-bacc7f1f151a',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 1024}}

## Smithery 에서 제공하는 MCP 서버

- 링크: https://smithery.ai/

사용한 도구 목록은 아래와 같습니다.

- Sequential Thinking: https://smithery.ai/server/@smithery-ai/server-sequential-thinking
  - 구조화된 사고 프로세스를 통해 역동적이고 성찰적인 문제 해결을 위한 도구를 제공하는 MCP 서버
- Desktop Commander: https://smithery.ai/server/@wonderwhy-er/desktop-commander
  - 다양한 편집 기능으로 터미널 명령을 실행하고 파일을 관리하세요. 코딩, 셸 및 터미널, 작업 자동화

**참고**

- smithery 에서 제공하는 도구를 JSON 형식으로 가져올때, 아래의 예시처럼 `"transport": "stdio"` 로 꼭 설정해야 합니다.

In [31]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent
from langchain_anthropic import ChatAnthropic

# LLM 모델 초기화
model = ChatAnthropic(model="claude-3-7-sonnet-latest", temperature=0, max_tokens=20000)

# 1. 클라이언트 생성
client = MultiServerMCPClient(
    {
        "server-sequential-thinking": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@smithery-ai/server-sequential-thinking",
                "--key",
                "89a4780a-53b7-4b7b-92e9-a29815f2669b",
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        },
        "desktop-commander": {
            "command": "npx",
            "args": [
                "-y",
                "@smithery/cli@latest",
                "run",
                "@wonderwhy-er/desktop-commander",
                "--key",
                "89a4780a-53b7-4b7b-92e9-a29815f2669b",
            ],
            "transport": "stdio",  # stdio 방식으로 통신을 추가합니다.
        },
        "document-retriever": {
            "command": "python",
            # mcp_server_rag.py 파일의 절대 경로로 업데이트해야 합니다
            "args": ["./mcp_server_rag.py"],
            # stdio 방식으로 통신 (표준 입출력 사용)
            "transport": "stdio",
        },
    }
)


# 2. 명시적으로 연결 초기화
await client.__aenter__()

langgraph 의 `create_react_agent` 를 사용하여 에이전트를 생성합니다.

In [32]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig


agent = create_react_agent(model, client.get_tools(), checkpointer=MemorySaver())

`Desktop Commander` 도구를 사용하여 터미널 명령을 실행합니다.

In [22]:
await astream_graph(
    agent,
    {
        "messages": "현재 경로를 포함한 하위 폴더 구조를 tree 로 그려줘. 단, .venv 폴더는 제외하고 출력해줘."
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
현재 경로와 하위 폴더 구조를 tree 형태로 그려드리겠습니다. `.venv` 폴더는 제외하고 출력하겠습니다.

이를 위해 `execute_command` 함수를 사용하여 tree 명령어를 실행하겠습니다. `-I` 옵션을 사용하면 특정 패턴을 제외할 수 있습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Command started with PID 79754
Initial output:
/bin/sh: tree: command not found

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
죄송합니다. 현재 시스템에 `tree` 명령어가 설치되어 있지 않은 것 같습니다. 대신 `find` 명령어를 사용하여 비슷한 결과를 얻을 수 있습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Command started with PID 79755
Initial output:
.
./.git
./.git/hooks
./.git/info
./.git/objects
./.git/objects/info
./.git/objects/pack
./.git/refs
./.git/refs/heads
./.git/refs/tags
./data

🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
이제 이 정보를 바탕으로 tree 형태로 시각화하겠습니다. 더 자세한 정보를 얻기 위해 파일도 함께 표시해 보겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
Command started with PID 

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-83c80294-4444-46ef-a194-3ec5f98f7ad7', usage_metadata={'input_tokens': 0, 'output_tokens': 292, 'total_tokens': 292, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 7,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:0b246418-d22a-48e1-bc24-ea4d16304163',
  'checkpoint_ns': 'agent:0b246418-d22a-48e1-bc24-ea4d16304163',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 1024}}

이번에는 `Sequential Thinking` 도구를 사용하여 비교적 복잡한 작업을 수행할 수 있는지 확인합니다.

In [33]:
await astream_graph(
    agent,
    {
        "messages": (
            "`retriever` 도구를 사용해서 삼성전자가 개발한 생성형 AI 관련 내용을 검색하고 "
            "`Sequential Thinking` 도구를 사용해서 보고서를 작성해줘."
        )
    },
    config=config,
)


🔄 Node: agent 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
삼성전자가 개발한 생성형 AI 관련 내용을 검색하고 보고서를 작성해 드리겠습니다. 먼저 `retrieve` 도구를 사용하여 관련 정보를 검색한 후, `sequentialthinking` 도구로 보고서를 체계적으로 작성하겠습니다.
🔄 Node: tools 🔄
- - - - - - - - - - - - - - - - - - - - - - - - - 
SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며, 
온디바이스에서 작동하도록 설계되어 외부로 사용자의 정보가 유출되지 않는 장점을 보유
∙삼성전자는 삼성 가우스를 활용한 온디바이스 AI 기술도 소개했으며, 생성 AI 모델을 다양한 제품에 
단계적으로 탑재할 계획
n 삼성 가우스는 △텍스트를 생성하는 언어모델 △코드를 생성하는 코드 모델 △이미지를 생성하는 
이미지 모델의 3개 모델로 구성
∙언어 모델은 클라우드와 온디바이스 대상 다양한 모델로 구성되며, 메일 작성, 문서 요약, 번

{'node': 'agent',
 'content': AIMessageChunk(content='', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-075ac28e-1e02-44b0-968e-c4f39becac2e', usage_metadata={'input_tokens': 0, 'output_tokens': 1807, 'total_tokens': 1807, 'input_token_details': {}}),
 'metadata': {'thread_id': 1,
  'langgraph_step': 21,
  'langgraph_node': 'agent',
  'langgraph_triggers': ('branch:to:agent', 'start:agent', 'tools'),
  'langgraph_path': ('__pregel_pull', 'agent'),
  'langgraph_checkpoint_ns': 'agent:610e1c41-7411-d1f4-6125-be63a47afddd',
  'checkpoint_ns': 'agent:610e1c41-7411-d1f4-6125-be63a47afddd',
  'ls_provider': 'anthropic',
  'ls_model_name': 'claude-3-7-sonnet-latest',
  'ls_model_type': 'chat',
  'ls_temperature': 0.0,
  'ls_max_tokens': 20000}}